In [14]:
# ONE-BUTTON NBA BUILDER + STRICT SCHEDULER (Jan 1 → Jun 30 + Finals by Jun 30)

from __future__ import annotations
import pandas as pd
from itertools import combinations
from collections import defaultdict
from datetime import datetime, timedelta
from pathlib import Path

# ========= YOUR EXACT PATHS =========
BASE = Path(r"C:\\Users\\Tanmay\\Downloads\\UNIVERSITY OF MELBOURNE STUDY FOLDER\\Scheduling and Optimisation")
TEAMS_CSV = BASE / "Teams_NBA.csv"
GAMES_CSV = BASE / "Games_NBA.csv"
OUT_SCHED = BASE / "Scheduled_NBA_Games_Strict.csv"

# ========= WINDOW + FINALS =========
SEASON_START = datetime(2025, 1, 1)
SEASON_END   = datetime(2025, 6, 30)

# set the Finals matchup here (home team has Games 1,2,5,7)
FINALS_HOME = "BOS"
FINALS_AWAY = "DEN"

# Finals dates (2-2-1-1-1) — guaranteed to finish by Jun 30; we will shift within Jun 20–30 if a per-team conflict occurs
FINALS_DATES = [
    datetime(2025, 6, 20),  # G1  home
    datetime(2025, 6, 22),  # G2  home
    datetime(2025, 6, 24),  # G3  away
    datetime(2025, 6, 26),  # G4  away
    datetime(2025, 6, 28),  # G5* home
    datetime(2025, 6, 29),  # G6* away
    datetime(2025, 6, 30),  # G7* home
]
FINALS_START_LOCAL = "20:00"
GAME_MINUTES = 150  # 2h30
def add_minutes(hhmm: str, minutes: int) -> str:
    h, m = map(int, hhmm.split(":"))
    dt = datetime(2000,1,1,h,m) + timedelta(minutes=minutes)
    return dt.strftime("%H:%M")
FINALS_END_LOCAL = add_minutes(FINALS_START_LOCAL, GAME_MINUTES)

# ========= 1) BUILD TEAMS_NBA.csv =========
E_ATL = ["BOS","BKN","NYK","PHI","TOR"]
E_CEN = ["CHI","CLE","DET","IND","MIL"]
E_SE  = ["ATL","CHA","MIA","ORL","WAS"]
W_NW  = ["DEN","MIN","OKC","POR","UTA"]
W_PAC = ["GSW","LAC","LAL","PHX","SAC"]
W_SW  = ["DAL","HOU","MEM","NOP","SAS"]

divisions = {
    "East": {"Atlantic":E_ATL, "Central":E_CEN, "Southeast":E_SE},
    "West": {"Northwest":W_NW, "Pacific":W_PAC, "Southwest":W_SW},
}

arenas = {
    "ATL": ("State Farm Arena","Atlanta"),
    "BOS": ("TD Garden","Boston"),
    "BKN": ("Barclays Center","Brooklyn"),
    "CHA": ("Spectrum Center","Charlotte"),
    "CHI": ("United Center","Chicago"),
    "CLE": ("Rocket Mortgage FieldHouse","Cleveland"),
    "DAL": ("American Airlines Center","Dallas"),
    "DEN": ("Ball Arena","Denver"),
    "DET": ("Little Caesars Arena","Detroit"),
    "GSW": ("Chase Center","San Francisco"),
    "HOU": ("Toyota Center","Houston"),
    "IND": ("Gainbridge Fieldhouse","Indianapolis"),
    "LAC": ("Crypto.com Arena","Los Angeles"),
    "LAL": ("Crypto.com Arena","Los Angeles"),
    "MEM": ("FedExForum","Memphis"),
    "MIA": ("Kaseya Center","Miami"),
    "MIL": ("Fiserv Forum","Milwaukee"),
    "MIN": ("Target Center","Minneapolis"),
    "NOP": ("Smoothie King Center","New Orleans"),
    "NYK": ("Madison Square Garden","New York"),
    "OKC": ("Paycom Center","Oklahoma City"),
    "ORL": ("Amway Center","Orlando"),
    "PHI": ("Wells Fargo Center","Philadelphia"),
    "PHX": ("Footprint Center","Phoenix"),
    "POR": ("Moda Center","Portland"),
    "SAC": ("Golden 1 Center","Sacramento"),
    "SAS": ("Frost Bank Center","San Antonio"),
    "TOR": ("Scotiabank Arena","Toronto"),
    "UTA": ("Delta Center","Salt Lake City"),
    "WAS": ("Capital One Arena","Washington, D.C."),
}

team_rows = []
for conf, divs in divisions.items():
    for dname, tlist in divs.items():
        for t in tlist:
            arena, city = arenas[t]
            team_rows.append({
                "team": t,
                "conference": "E" if conf=="East" else "W",
                "division": dname,
                "arena": arena,
                "city": city
            })

teams_df = pd.DataFrame(team_rows).sort_values("team").reset_index(drop=True)
TEAMS_CSV.parent.mkdir(parents=True, exist_ok=True)
teams_df.to_csv(TEAMS_CSV, index=False)

# ========= 2) BUILD FULL 1,230-GAME GAMES_NBA.csv =========
teams = teams_df["team"].tolist()

def add_series(glist, home, away, n, split=None):
    if split is None:
        assert n % 2 == 0
        h = n // 2
        for _ in range(h): glist.append((home, away))
        for _ in range(h): glist.append((away, home))
    else:
        h1, h2 = split
        for _ in range(h1): glist.append((home, away))
        for _ in range(h2): glist.append((away, home))

games = []

# Division: 4 per pair (2H/2A)
for conf, divs in divisions.items():
    for dname, tlist in divs.items():
        for a, b in combinations(tlist, 2):
            add_series(games, a, b, 4)

# Interconference: 2 per pair (1H/1A)
east_all = sum(divisions["East"].values(), [])
west_all = sum(divisions["West"].values(), [])
for e in east_all:
    for w in west_all:
        add_series(games, e, w, 2)

# Conference (cross-division): circulant template — each team gets 6 opponents x4 and 4 opponents x3 (balanced 2H/1A)
def add_cross_div(glist, A, B):
    n = 5
    for i, a in enumerate(A):
        for j, b in enumerate(B):
            off = (j - i) % n
            if off in (0,1,3):
                add_series(glist, a, b, 4)      # 2H/2A
            else:
                # three-game series: offset 2 -> A gets 2H/1A; offset 4 -> B gets 2H/1A
                if off == 2:
                    add_series(glist, a, b, 3, split=[2,1])
                elif off == 4:
                    add_series(glist, b, a, 3, split=[2,1])
                else:
                    raise RuntimeError("Unexpected offset")

# East cross-division
add_cross_div(games, E_ATL, E_CEN)
add_cross_div(games, E_ATL, E_SE)
add_cross_div(games, E_CEN, E_SE)
# West cross-division
add_cross_div(games, W_NW, W_PAC)
add_cross_div(games, W_NW, W_SW)
add_cross_div(games, W_PAC, W_SW)

# Verify totals
home_ct = defaultdict(int); away_ct = defaultdict(int)
for h, a in games:
    home_ct[h] += 1; away_ct[a] += 1
assert len(games) == 1230, f"Expected 1230, got {len(games)}"
assert all(home_ct[t] == 41 and away_ct[t] == 41 for t in teams), "41H/41A balance failed"

games_df = pd.DataFrame(
    [{"game_id": f"G{idx:04d}", "home": h, "away": a} for idx, (h, a) in enumerate(games, start=1)]
)
games_df.to_csv(GAMES_CSV, index=False)

# ========= 3) STRICT SCHEDULER (Jan 1 – Jun 30), no team twice/day =========
SLOTS = ["12:00","13:30","15:00","16:30","18:00","19:30","21:00"]  # plenty of same-day capacity
def add_minutes(hhmm: str, minutes: int) -> str:
    h, m = map(int, hhmm.split(":"))
    dt = datetime(2000,1,1,h,m) + timedelta(minutes=minutes)
    return dt.strftime("%H:%M")

all_dates = [SEASON_START + timedelta(days=i) for i in range((SEASON_END - SEASON_START).days + 1)]
date_strs = [d.strftime("%Y-%m-%d") for d in all_dates]

team_to_venue = {r.team: r.arena for r in teams_df.itertuples(index=False)}

team_busy_day = set()     # (team, date_str)
slots_used = defaultdict(int)
rows = []

# Greedy earliest-fit: allows many league games per day, blocks per-team day collisions
for gid, home, away in games_df[["game_id","home","away"]].itertuples(index=False):
    placed = False
    for ds in date_strs:
        if (home, ds) in team_busy_day or (away, ds) in team_busy_day:
            continue
        idx = slots_used[ds] % len(SLOTS)
        st = SLOTS[idx]
        et = add_minutes(st, GAME_MINUTES)
        rows.append({
            "game_id": gid, "home": home, "away": away,
            "date": ds, "start_time_local": st, "end_time_local": et,
            "venue": team_to_venue.get(home, f"{home} Arena")
        })
        team_busy_day.add((home, ds)); team_busy_day.add((away, ds))
        slots_used[ds] += 1
        placed = True
        break
    if not placed:
        raise RuntimeError(f"Could not place {gid} ({home} vs {away}) within Jan 1–Jun 30 under per-team/day constraint.")

# ========= 4) APPEND FINALS (best-of-7, finish by Jun 30) =========
final_homes = [FINALS_HOME, FINALS_HOME, FINALS_AWAY, FINALS_AWAY, FINALS_HOME, FINALS_AWAY, FINALS_HOME]
final_away  = [FINALS_AWAY if h==FINALS_HOME else FINALS_HOME for h in final_homes]

# try each planned date; if either team is already busy, search within Jun 20–30 for nearest free date
for i, planned_dt in enumerate(FINALS_DATES, start=1):
    h = final_homes[i-1]
    a = final_away[i-1]
    # search window
    candidates = [planned_dt + timedelta(days=k) for k in range(0, 1)] + \
                 [planned_dt - timedelta(days=1), planned_dt + timedelta(days=1),
                  planned_dt - timedelta(days=2), planned_dt + timedelta(days=2),
                  planned_dt - timedelta(days=3), planned_dt + timedelta(days=3),
                  planned_dt - timedelta(days=4), planned_dt + timedelta(days=4),
                  planned_dt - timedelta(days=5), planned_dt + timedelta(days=5)]
    # restrict to Jun 20–30
    candidates = [d for d in candidates if datetime(2025,6,20) <= d <= datetime(2025,6,30)]
    chosen_ds = None
    for d in candidates:
        ds = d.strftime("%Y-%m-%d")
        if (h, ds) not in team_busy_day and (a, ds) not in team_busy_day:
            chosen_ds = ds
            break
    if chosen_ds is None:
        # absolute last resort: scan Jun 20–30 for any free day
        for d in (datetime(2025,6,20) + timedelta(days=k) for k in range(11)):
            ds = d.strftime("%Y-%m-%d")
            if (h, ds) not in team_busy_day and (a, ds) not in team_busy_day:
                chosen_ds = ds
                break
        if chosen_ds is None:
            raise RuntimeError("Could not place a Finals game within Jun 20–30 without same-day double-booking a team.")

    rows.append({
        "game_id": f"F{i}", "home": h, "away": a,
        "date": chosen_ds, "start_time_local": FINALS_START_LOCAL, "end_time_local": FINALS_END_LOCAL,
        "venue": team_to_venue.get(h, f"{h} Arena")
    })
    team_busy_day.add((h, chosen_ds)); team_busy_day.add((a, chosen_ds))

# ========= 5) SAVE SCHEDULE =========
sched_df = pd.DataFrame(rows).sort_values(["date","start_time_local","game_id"]).reset_index(drop=True)
OUT_SCHED.parent.mkdir(parents=True, exist_ok=True)
sched_df.to_csv(OUT_SCHED, index=False)

print("Done!")
print(f" - Teams file:     {TEAMS_CSV}")
print(f" - Games file:     {GAMES_CSV}  (rows={len(games_df)})")
print(f" - Schedule file:  {OUT_SCHED}  (rows={len(sched_df)})")


Done!
 - Teams file:     C:\Users\Tanmay\Downloads\UNIVERSITY OF MELBOURNE STUDY FOLDER\Scheduling and Optimisation\Teams_NBA.csv
 - Games file:     C:\Users\Tanmay\Downloads\UNIVERSITY OF MELBOURNE STUDY FOLDER\Scheduling and Optimisation\Games_NBA.csv  (rows=1230)
 - Schedule file:  C:\Users\Tanmay\Downloads\UNIVERSITY OF MELBOURNE STUDY FOLDER\Scheduling and Optimisation\Scheduled_NBA_Games_Strict.csv  (rows=1237)
